In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key
%matplotlib notebook

# 1. Load Data

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/WeatherPy_challenge.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description,Rain Inches (last 3 hrs),Snow Inches (last 3 hrs),Date
0,0,Thompson,55.74,-97.86,14.00,66,90,5.82,CA,overcast clouds,0.00,0.0,2020-02-23 18:40:10
1,1,Busselton,-33.65,115.33,68.00,44,93,15.14,AU,light rain,0.69,0.0,2020-02-23 18:40:44
2,2,Rio Grande,-32.03,-52.10,78.80,47,20,17.22,BR,few clouds,0.00,0.0,2020-02-23 18:49:01
3,3,Rikitea,-23.12,-134.97,78.37,74,2,8.25,PF,light rain,0.44,0.0,2020-02-23 18:40:23
4,4,Mataura,-46.19,168.86,51.01,92,93,3.00,NZ,overcast clouds,0.00,0.0,2020-02-23 18:40:31


In [3]:
city_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 557 entries, 0 to 556
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   City_ID                   557 non-null    int64  
 1   City                      557 non-null    object 
 2   Lat                       557 non-null    float64
 3   Lng                       557 non-null    float64
 4   Max Temp                  557 non-null    float64
 5   Humidity                  557 non-null    int64  
 6   Cloudiness                557 non-null    int64  
 7   Wind Speed                557 non-null    float64
 8   Country                   553 non-null    object 
 9   Current Description       557 non-null    object 
 10  Rain Inches (last 3 hrs)  557 non-null    float64
 11  Snow Inches (last 3 hrs)  557 non-null    float64
 12  Date                      557 non-null    object 
dtypes: float64(6), int64(3), object(4)
memory usage: 56.7+ KB


# 2.  Develop Dataframe Based on Criteria

In [4]:
#Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
# Ask the customer to add a rain and snow values
Rain= input("Do you want it to be raining? (yes/no)?")
Snow= input("Do you want it to be snowing? (yes/no)?")

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90
Do you want it to be raining? (yes/no)?yes
Do you want it to be snowing? (yes/no)?no


In [5]:
# filter the dataset to find the cities
preferred_cities_df=city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]

if Rain=="no" and Snow=="no":
    preferred_cities_df=preferred_cities_df.loc[(preferred_cities_df["Rain Inches (last 3 hrs)"]<=0) & (preferred_cities_df["Snow Inches (last 3 hrs)"]<=0)]                                                                          

elif Rain=="no":
    preferred_cities_df=preferred_cities_df.loc[(preferred_cities_df["Rain Inches (last 3 hrs)"]<=0),:]
                                                 
elif Snow=="no":
    preferred_cities_df=preferred_cities_df.loc[(preferred_cities_df["Snow Inches (last 3 hrs)"]<=0),:]                                             

else:
    preferred_cities_df

preferred_cities_df.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 230 entries, 2 to 555
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   City_ID                   230 non-null    int64  
 1   City                      230 non-null    object 
 2   Lat                       230 non-null    float64
 3   Lng                       230 non-null    float64
 4   Max Temp                  230 non-null    float64
 5   Humidity                  230 non-null    int64  
 6   Cloudiness                230 non-null    int64  
 7   Wind Speed                230 non-null    float64
 8   Country                   229 non-null    object 
 9   Current Description       230 non-null    object 
 10  Rain Inches (last 3 hrs)  230 non-null    float64
 11  Snow Inches (last 3 hrs)  230 non-null    float64
 12  Date                      230 non-null    object 
dtypes: float64(6), int64(3), object(4)
memory usage: 25.2+ KB


In [6]:
# drop null rows
preferred_cities_df=preferred_cities_df.dropna()

In [7]:
preferred_cities_df.count()

City_ID                     229
City                        229
Lat                         229
Lng                         229
Max Temp                    229
Humidity                    229
Cloudiness                  229
Wind Speed                  229
Country                     229
Current Description         229
Rain Inches (last 3 hrs)    229
Snow Inches (last 3 hrs)    229
Date                        229
dtype: int64

# 3.  Get Travel Data Information

In [8]:
# Print results rain and snow count
rain_count=preferred_cities_df.loc[preferred_cities_df["Rain Inches (last 3 hrs)"]>0, ["Rain Inches (last 3 hrs)"]].count()
snow_count=preferred_cities_df.loc[preferred_cities_df["Snow Inches (last 3 hrs)"]>0, ["Snow Inches (last 3 hrs)"]].count()
print(f'rain count:{rain_count},snow count:{snow_count}')

rain count:Rain Inches (last 3 hrs)    48
dtype: int64,snow count:Snow Inches (last 3 hrs)    0
dtype: int64


In [10]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp","Current Description", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Rio Grande,BR,78.80,few clouds,-32.03,-52.10,
3,Rikitea,PF,78.37,light rain,-23.12,-134.97,
8,Fare,PF,82.72,scattered clouds,-16.70,-151.02,
10,Ixtapa,MX,80.60,scattered clouds,20.70,-105.20,
13,Cayenne,GF,84.20,broken clouds,4.93,-52.33,


In [11]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [12]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [13]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Rio Grande,BR,78.80,few clouds,-32.03,-52.10,Swan Express Rio Grande
3,Rikitea,PF,78.37,light rain,-23.12,-134.97,Pension Maro'i
8,Fare,PF,82.72,scattered clouds,-16.70,-151.02,Maitai Lapita Village Huahine
10,Ixtapa,MX,80.60,scattered clouds,20.70,-105.20,Container Inn Puerto Vallarta
13,Cayenne,GF,84.20,broken clouds,4.93,-52.33,Hôtel Le Dronmi


In [14]:
hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 229 entries, 2 to 555
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City                 229 non-null    object 
 1   Country              229 non-null    object 
 2   Max Temp             229 non-null    float64
 3   Current Description  229 non-null    object 
 4   Lat                  229 non-null    float64
 5   Lng                  229 non-null    float64
 6   Hotel Name           229 non-null    object 
dtypes: float64(3), object(4)
memory usage: 24.3+ KB


In [22]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>
"""

In [23]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [24]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=100,point_radius=1)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [25]:
# Create the output file (CSV).
output_data_file = "weather_data/WeatherPy_vacation.csv"
# Export the City_Data into a CSV.
hotel_df.to_csv(output_data_file, index_label="City_ID")

End of File